In [1]:
### import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint
from pkg.utils import plot_timeseries

# Model - Waning

Here we build off the modified SEIR model to include the possible waning effect of vaccination. Protection provided by vaccination is known to decrease over time, causing the vaccinated population to become once again susceptible to the disease. Thus, it is important to explore the effect of waning on the population dynamics between the two vaccine failure modes. Similar to the other models, all variables were normalized such that $N = S + V_{S} + V_{R} + E + I + R = 1$ and vaccines were administered instantaneously.

Also for simplcity, we consider vaccinated individuals with waned immunity to be equally as susceptible to the disease as unvaccinated individuals, and we assume that waning happens instantaneously. 

https://www.sciencedirect.com/science/article/pii/S1877750319303163

### Equations
\begin{align*}
\dot{S} &= - \beta SI - v(t)S + w(t)(V_{S} + V_{R})\\
\dot{V}_{S} &= (1 - \epsilon_A)v(t)S - \beta V_{S}I - w(t)V_{S}\\
\dot{V}_{R} &= \epsilon_A v(t)S - \beta(1-\epsilon_L)V_{R}I + w(t)V_{R}\\
\dot{E} &= \beta (S + V_S + (1 - \epsilon_L) V_R)I - \sigma E \\
\dot{I} &= \sigma E - \gamma I \\
\dot{R} &= \gamma I \\
\end{align*}

### Variables
- $S$: Fraction of susceptible and uninfected individuals
- $V_{S}$: Fraction of vaccinated and uninfected individuals who are equally as susceptible as unvaccinated individuals 
- $V_{R}$: Fraction of vaccinated and uninfected individuals who are less susceptible than unvaccinated individuals
- $E$: Fraction of exposed individuals who are not yet infected
- $I$: Fraction of infected individuals
- $R$: Fraction of recovered individuals

### Parameters
- $\beta$: Infectiousnes of infected individuals
- $v(t)$: Fraction of population vaccinated per unit time. For an instantaneous vaccination campaign at time $t_V$ that targets a fraction $f_V$ of the population, $v(t) = f_V \delta (t - t_V)$ where $\delta$ is the Kronecker delta function. For a continuous vaccine rollout beginning at time $t_V$ at initial rate $v_{max}$ and continuing until a fraction $f_V$ of the population are vaccinated, $v(t) = v_{max}H(t-t_V)(1 - \frac{V_S + V_R}{f_V})$ where $H$ is the Heaviside step function.
- $\epsilon_A$: Efficacy of 'all-or-nothing' protection
- $\epsilon_L$: Efficacy of 'leaky' protection
- $\sigma$: Duration of the latent period
- $\gamma$: Duration of the infectious period
- $w(t)$: Fraction of vaccinated population with waned immunity per unit time. For an instantaneous waning effect at time $t_W$ that targets a fraction $f_W$ of the population, $w(t) = f_W \delta (t - t_W)$ where $\delta$ is the Kronecker delta function. For a continuous waning effect beginning at time $t_W$ at initial rate 0 and continuing until $V_S + V_R$ becomes 0 at maximum rate $w_{max}, $w(t) = w_{max}H(t-t_W)(1 - \frac{V_S + V_R}{f_V})$ where $H$ is the Heaviside step function and $f_V$ is the fraction of the population that are vaccinated.